In [2]:
# workspace/notebooks/gpu/quick_rocm_resnet_test.ipynb

import torch
import torch.nn as nn
import sys
import os

# Add path to your rocm_resnet.py
sys.path.append('/workspace/notebooks/pylibs')

try:
    from rocm_resnet import rocm_resnet18
    print("✓ Imported ROCm ResNet18")
except ImportError as e:
    print(f"✗ Import error: {e}")
    
# Set ROCm environment variables
os.environ['MIOPEN_DISABLE_CACHE'] = '1'

if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print(f"Device: {torch.cuda.get_device_name(0)}")
    
    # Create model
    model = rocm_resnet18(num_classes=10).to(device)
    print(f"Model created with {sum(p.numel() for p in model.parameters()):,} parameters")
    
    # Test forward pass
    batch_size = 64
    inputs = torch.randn(batch_size, 3, 32, 32, device=device)
    
    print(f"\nTesting forward pass with batch_size={batch_size}...")
    
    try:
        # Set to eval mode
        model.eval()
        
        with torch.no_grad():
            outputs = model(inputs)
        
        print(f"✓ Forward pass successful!")
        print(f"  Input shape: {inputs.shape}")
        print(f"  Output shape: {outputs.shape}")
        print(f"  Memory used: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
        
        # Test backward pass
        print(f"\nTesting backward pass...")
        model.train()
        
        targets = torch.randint(0, 10, (batch_size,), device=device)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        print(f"✓ Backward pass successful!")
        print(f"  Loss: {loss.item():.6f}")
        print(f"  Peak memory: {torch.cuda.max_memory_allocated() / 1e9:.2f} GB")
        
    except Exception as e:
        print(f"✗ Error: {type(e).__name__}: {e}")
    
    print(f"\n✓ ROCm ResNet test completed without MIOpen errors!")
    
else:
    print("No GPU available!")

✓ Imported ROCm ResNet18
Device: AMD Radeon Graphics
Model created with 11,173,962 parameters

Testing forward pass with batch_size=64...
✓ Forward pass successful!
  Input shape: torch.Size([64, 3, 32, 32])
  Output shape: torch.Size([64, 10])
  Memory used: 0.13 GB

Testing backward pass...
✓ Backward pass successful!
  Loss: 2.289127
  Peak memory: 0.53 GB

✓ ROCm ResNet test completed without MIOpen errors!
